# Code Assignment 1
Maximilian Huber

In [1]:
# setup functions
using Optim, Plots, ForwardDiff, Roots; gr()

u(x, z) = -10 * convert(Float64, norm(x - z) >= 0.1)
c(y, z) = norm(y - z)^2

function W(p)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    return sum(n_x .* log.(1 .+ sum(exp.(U .- p'), 2))) + sum(m_y .* log.(1 .+ sum(exp.(p' .- C), 2)))
end

D(p, U) = sum(n_x .* exp.(U .- p') ./ (1 .+ sum(exp.(U .- p'), 2)), 1)[:]
S(p, C) = sum(m_y .* exp.(p' .- C) ./ (1 .+ sum(exp.(p' .- C), 2)), 1)[:]

function E!(e, U, C, p)
    e .= S(p, C) .- D(p, U)
    return nothing
end;

function E_z(U, C, p, z)
    S(p, C)[z] - D(p, U)[z]
end

function E(U, C, p)
    S(p, C) - D(p, U)
end

E (generic function with 1 method)

In [2]:
# define X, Y, Z, N, n_x, m_y
const X = [[i, j] for i in 0.1:0.1:1, j in 0.1:0.1:1][:]
const Y = X
const Z = X
const N = 100

const n_x = ones(N)
const m_y = prod.(Y);

## Gradient Descent

In [3]:
# define gradient_decent()
function gradient_decent(;stepsize = 0.2, tol = 1e-5, max_iter = 500, p_init = zeros(N))
    tic()
    iteration = 0
    residual = 1.
    p_old = p_init
    p_new = zeros(N)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    while (residual > tol) && (iteration <= max_iter)
        iteration += 1
        E!(p_new, U, C, p_old)
        p_new .= p_old .- stepsize .* p_new

        residual = maximum(abs.(S(p_new, C) .- D(p_new, U)) ./
            (S(p_new, C) .+ D(p_new, U)))
        p_old .= p_new
    end
    
    return (residual, sum(S(p_new, C)), 
        sum(S(p_new, C) .* p_new), iteration, toq(), p_new)
end

gradient_decent (generic function with 1 method)

Off course, the choice of the `stepsize=2.0` is optimized for this concrete example!

In [4]:
(residual, rides, avg_price, iter, t, p) = gradient_decent(stepsize=2.0)

@show (residual, rides, avg_price, iter, t);

(residual, rides, avg_price, iter, t) = (6.6792805490928935e-6, 30.196710381492267, 61.64504758321407, 24, 0.035827966)


In [5]:
p_correct = p;

## Implicit Time Stepping

In [6]:
function residual_wrapper(p_old, stepsize, U, C)
    return (residual, p_new) -> residual .= E(U, C, p_new) - 1/stepsize * (p_new - p_old)
end

residual_wrapper (generic function with 1 method)

In [7]:
# define gradient_decent()
function implicit_time_stepping(;stepsize = 1., tol = 1e-5, max_iter = 50, p_init = ones(N))
    tic()
    iteration = 0
    distance = 1.
    distanceold = 1.
    p_old = p_init
    p_new = zeros(N)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    while (distance > tol) && (iteration <= max_iter)
        iteration += 1
        result = nlsolve(residual_wrapper(p_old, stepsize, U, C), p_old, iterations = 100, 
            autodiff=:forward, method = :newton, ftol = 1e-9)
        p_new .= result.zero
        
        if any(isnan.(p_new))
            println("Iteration $iteration bad solution with step size $stepsize")
            stepsize = stepsize / 10
            continue
        end
        
        if !result.f_converged
            println("Iteration $iteration no convergence with step size $stepsize")
            stepsize = stepsize / 10
            continue
        end      
        
        distance, distanceold = maximum(abs.(E(U, C, p_new))), distance
        p_old .= p_new
        if (distance <= distanceold)
            stepsize = stepsize * 2
        end
    end
    
    return (distance, sum(S(p_new, C)), 
        (S(p_new, C)' * p_new)[1]/sum(S(p_new, C)), iteration, toq(), p_new)
end

implicit_time_stepping (generic function with 1 method)

In [8]:
using NLsolve

(residual, rides, avg_price, iter, t, p) = implicit_time_stepping(stepsize=0.1, max_iter = 20)

@show (residual, rides, avg_price, iter, t);

(residual, rides, avg_price, iter, t) = (100.0, 30.249999999999915, 33.532630609023045, 21, 5.916749272)


In [9]:
maximum(abs.(p .- p_correct))

71.22259540764466

In [36]:
U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
(E(U, C, p))[89]

-100.0

## Newton Descent

In [5]:
# define newton_decent()
function newton_decent(;stepsize = .2, tol = 1e-5, max_iter = 50, p_init = zeros(N))
    tic()
    iteration = 0
    residual = 1.
    p_old = p_init
    p_new = zeros(N)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    f = OnceDifferentiable((e, p) -> E!(e, U, C, p), zeros(100))
    df = zeros(100, 100)
    
    while (residual > tol) && (iteration <= max_iter)
        iteration += 1
        E!(p_new, U, C, p_old)
        p_new .= p_old .- stepsize .* ((ForwardDiff.hessian(W, p_old)^-1) * p_new)

        residual = maximum(abs.(S(p_new, C) .- D(p_new, U)) ./
            (S(p_new, C) .+ D(p_new, U)))
        p_old .= p_new
    end
    
    return (residual, sum(S(p_new, C)), 
        sum(S(p_new, C) .* p_new), iteration, toq(), p_new)
end

newton_decent (generic function with 1 method)

In [6]:
#(residual, rides, avg_price, iter, t, p) = newton_decent(stepsize = .2)

#@show (residual, rides, avg_price, iter, t);

## Coordinate Decent

In [15]:
# define coordinate_decent()
function coordinate_decent(;tol::Float64 = 1e-5, max_iter::Int64 = 50, p_init::Vector{Float64} = zeros(N))
    tic()
    iteration = 0
    residual = 1.
    p_old = p_init
    p_new = zeros(N)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    while (residual > tol) && (iteration < max_iter)
        iteration += 1

        Threads.@threads for i in 1:N
            p_new[i] = find_zero(p_z -> E_z(U, C, vcat(p_old[1:i-1], p_z, p_old[i+1:end]), i), (0., 3.), Order1())
        end
        
        residual = maximum(abs.(S(p_new, C) .- D(p_new, U)) ./ (S(p_new, C) .+ D(p_new, U)))
        p_old .= p_new
    end
    
    return (residual, sum(S(p_new, C)), 
        (S(p_new, C)' * p_new)[1]/sum(S(p_new, C)), iteration, toq(), p_new)
end

coordinate_decent (generic function with 1 method)

In [16]:
(residual, rides, avg_price, iter, t, p) = coordinate_decent()

@show (residual, rides, avg_price, iter, t);

(residual, rides, avg_price, iter, t) = (6.527430780612634e-6, 30.19670964825541, 2.0414353028463283, 27, 3.679892047)


In [9]:
# define coordinate_decent()
function coordinate_decent_single(;tol::Float64 = 1e-5, max_iter::Int64 = 50, p_init::Vector{Float64} = zeros(N))
    tic()
    iteration = 0
    residual = 1.
    p_old = p_init
    p_new = zeros(N)
    
    U = zeros(N, N)
    C = zeros(N, N)
    
    for i in 1:N # passanger/car locations
        for j in 1:N #pickup locations
            U[i, j] = u(X[i], Z[j])
            C[i, j] = c(Y[i], Z[j])
        end
    end
    
    while (residual > tol) && (iteration < max_iter)
        iteration += 1

        for i in 1:N
            p_new[i] = find_zero(p_z -> E_z(U, C, vcat(p_old[1:i-1], p_z, p_old[i+1:end]), i), (0., 3.), Order1())
        end
        
        residual = maximum(abs.(S(p_new, C) .- D(p_new, U)) ./ (S(p_new, C) .+ D(p_new, U)))
        p_old .= p_new
    end
    
    return (residual, sum(S(p_new, C)), 
        (S(p_new, C) * p_new)[1]/sum(S(p_new, C)), iteration, toq(), p_new)
end

coordinate_decent_single (generic function with 1 method)

In [10]:
(residual, rides, avg_price, iter, t, p) = coordinate_decent_single()

@show (residual, rides, avg_price, iter, t);

(residual, rides, avg_price, iter, t) = (6.527430780612634e-6, 30.19670964825541, 2.0414353028463283, 27, 10.287812548)


## Subsidizer Supply

In [11]:
F(p) = p .+ log.(1 .+ exp.(-p))
S(p, C) = sum(m_y .* exp.(F(p)' .- C) ./ (1 .+ sum(exp.(F(p)' .- C), 2)), 1)

S (generic function with 1 method)

In [12]:
(residual, rides, avg_price, iter, t, p) = coordinate_decent(max_iter = 200)

@show (residual, rides, avg_price, iter, t);

(residual, rides, avg_price, iter, t) = (8.324873739471545e-6, 30.202834872610644, 2.0407585942787714, 24, 4.37938677)


The coordinate descent method is inefficient, when faced with the subsidized supply function.